In [2]:
import xml.etree.ElementTree as ET
import datetime
import time
import sys
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.error import HTTPError



In [3]:
newURL = '{http://arxiv.org/OAI/arXiv/}'
W3 = '{http://www.w3.org/2005/Atom}'
A9 = '{http://a9.com/-/spec/opensearch/1.1/}'
myBASE = 'http://export.arxiv.org/api/query?'

In [4]:
url = 'http://export.arxiv.org/api/query?search_query=all:network'
response = urlopen(url)
xml = response.read()

In [5]:
root = ET.fromstring(xml)
records = root.findall(W3 + 'entry')

In [6]:
root.tag

'{http://www.w3.org/2005/Atom}feed'

In [7]:
print(records)

[<Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a5285e8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a528bd8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536368>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536908>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536e08>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a0984a8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a098bd8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a09d368>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a09dc28>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a0a64f8>]


In [ ]:
for record in records:
  meta = record.find(W3 + 'summary')
  